Dealing with Data That Lives in Memory
When working with large datasets that reside in memory, efficient management and processing become crucial. The vertexai library and SDK provide essential tools for managing data in the cloud. A data warehouse, acting as a central repository, aggregates data from various sources, including application databases, files, and other external data sources. BigQuery, as a serverless data warehouse, allows SQL queries to be used for processing vast amounts of data efficiently.

Stack Overflow Dataset
The Stack Overflow dataset serves as an illustrative example of such large datasets. It contains tables related to questions, answers, and associated metadata. Users can query and fetch data from this dataset through a notebook interface, writing SQL queries to retrieve data for analysis or fine-tuning machine learning models.

Dealing with Large Datasets in Memory
Large datasets often cannot fit into memory all at once. SQL queries can be constrained or limited to ensure that the data fits into memory. A 403 error may occur if the dataset exceeds memory limits or access is restricted. Best practices to mitigate this include using data lineage, which tracks the movement and transformation of data. For more efficient storage and access, cloud storage buckets or SSDs can be employed. Another effective solution is data streaming, which processes data in smaller, more manageable batches.

Instructions to Large Language Models (LLMs)
When providing instructions to Large Language Models (LLMs), specific directives are given to help the model understand the task. For example, the model may be instructed to answer a Stack Overflow question. Rather than looping through the entire dataset, it is more efficient to use a labelled unique ID that allows the model to jump directly to the relevant data. This process improves efficiency, allowing the parser to quickly identify similar labels and produce the corresponding output.

Data Formats for Large Datasets
Different formats are used to handle large datasets effectively:

JSONL (JSON Lines): This format is often used when storing data as rows, such as question-answer pairs. It is easy to parse and widely used in machine learning applications.
TFRecord: A binary format designed for efficient reading and writing. It is particularly well-suited for large-scale datasets and is commonly used with TensorFlow models. Its binary nature allows for faster reading and lower memory overhead, making it ideal for machine learning tasks involving large amounts of data.
Parquet: A columnar storage format that is highly efficient for complex and large datasets. It provides better performance for reading, querying, and compressing data compared to simpler formats like CSV.
Versioning and Artifacts
In machine learning workflows, versioning artifacts is essential for reproducibility and traceability. Versioning datasets, models, and other artifacts ensures that the entire process can be repeated or traced back to a specific state. This practice is critical for maintaining control over the different iterations of data, models, and experiments, allowing for robust model development and deployment strategies.


Step 1: Project Environment Setup
Load Credentials and Relevant Python Libraries
If you were running this notebook locally, you would first install Vertex AI.


In [ ]:
# The necessary Vertex AI SDK can be installed with the following line if required
# !pip install google-cloud-aiplatform
from utils import authenticate
credentials, PROJECT_ID = authenticate()  # Authenticate and get credentials and project ID
REGION = "us-central1"  # Set the region for Vertex AI services


Step 2: Import Vertex AI SDK and Initialize
Import the Vertex AI SDK to interact with the Vertex AI services in the cloud.
Initialize the Vertex AI SDK with the project ID, region, and credentials.

In [ ]:
import vertexai
vertexai.init(project=PROJECT_ID, 
              location=REGION, 
              credentials=credentials)  # Initialize Vertex AI SDK


Step 3: Import and Initialize BigQuery Client
Import BigQuery to use as your data warehouse.
Initialize the BigQuery client to start querying and interacting with the data.

In [ ]:
from google.cloud import bigquery
bq_client = bigquery.Client(project=PROJECT_ID, 
                            credentials=credentials)  # Initialize BigQuery client


Step 4: Stack Overflow Public Dataset Exploration
4.1: Querying for Table Names
Use SQL to retrieve the names of all tables from the Stack Overflow public dataset.
You will send the query using the BigQuery client and retrieve table names.

In [ ]:
QUERY_TABLES = """
SELECT
  table_name
FROM
  `bigquery-public-data.stackoverflow.INFORMATION_SCHEMA.TABLES`
"""

# Execute the query and print the table names
query_job = bq_client.query(QUERY_TABLES)
for row in query_job:
    for value in row.values():
        print(value)


4.2: Data Retrieval
Fetch some data from the data warehouse and store it in a Pandas DataFrame for easier exploration and visualization.
Here we will retrieve 3 rows from the posts_questions table.

In [ ]:
INSPECT_QUERY = """
SELECT
    *
FROM
    `bigquery-public-data.stackoverflow.posts_questions`
LIMIT 3
"""

import pandas as pd
# Execute the query and load results into a Pandas DataFrame
query_job = bq_client.query(INSPECT_QUERY)
stack_overflow_df = query_job.result().to_arrow().to_pandas()  # Convert Arrow table to Pandas DataFrame
stack_overflow_df.head()  # Show the first few rows


Step 5: Dealing with Large Datasets
For large datasets that cannot fit into memory, we handle them carefully by processing them in smaller batches or optimizing queries.

In [ ]:
QUERY_ALL = """
SELECT
    *
FROM
    `bigquery-public-data.stackoverflow.posts_questions` q
"""

query_job = bq_client.query(QUERY_ALL)
try:
    stack_overflow_df = query_job.result().to_arrow().to_pandas()  # This will raise an exception if too large
except Exception as e:
    print('The DataFrame is too large to load into memory.', e)


Step 6: Joining Tables and Query Optimization
6.1: SQL Query to Join Posts and Answers
We select questions and answers from the posts_questions and posts_answers tables.
Join them based on the unique question ID, filter for Python-related questions, and limit the results to 10,000.

In [ ]:
QUERY = """
SELECT
    CONCAT(q.title, q.body) as input_text,
    a.body AS output_text
FROM
    `bigquery-public-data.stackoverflow.posts_questions` q
JOIN
    `bigquery-public-data.stackoverflow.posts_answers` a
ON
    q.accepted_answer_id = a.id
WHERE
    q.accepted_answer_id IS NOT NULL AND
    REGEXP_CONTAINS(q.tags, "python") AND
    a.creation_date >= "2020-01-01"
LIMIT
    10000
"""

# Execute the optimized query to retrieve data
query_job = bq_client.query(QUERY)
stack_overflow_df = query_job.result().to_arrow().to_pandas()  # Store results in Pandas DataFrame
stack_overflow_df.head(2)  # Display the first two rows


Step 7: Adding Instructions for LLM
To improve the performance of the Language Model, we add instructions to the input questions.
This helps the model understand what task to perform (i.e., answering a Stack Overflow question).

In [ ]:
# Instruction template to guide the LLM for better generalization
INSTRUCTION_TEMPLATE = """\
Please answer the following Stackoverflow question on Python. \
Answer it like you are a developer answering Stackoverflow questions.

Stackoverflow question:
"""

# Combine the instruction template with the input text (questions)
stack_overflow_df['input_text_instruct'] = INSTRUCTION_TEMPLATE + ' ' + stack_overflow_df['input_text']


Step 8: Dataset for Tuning
8.1: Splitting Data into Training and Evaluation Sets
Divide the data into 80% for training and 20% for evaluation using train_test_split.
The random_state ensures the split is reproducible.

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training (80%) and evaluation (20%) sets
train, evaluation = train_test_split(
    stack_overflow_df,
    test_size=0.2,
    random_state=42
)


Step 9: Generating JSONL Files for Training
9.1: Data Versioning
Versioning is important for reproducibility.
Generate a unique file name using the current timestamp.

In [ ]:
import datetime
date = datetime.datetime.now().strftime("%H:%d:%m:%Y")  # Get current date and time


9.2: Creating JSONL File for Model Tuning
Save the input questions (with instructions) and corresponding answers in JSONL format for easy use during tuning.


In [ ]:
cols = ['input_text_instruct','output_text']  # Columns to include in the JSONL file
tune_jsonl = train[cols].to_json(orient="records", lines=True)  # Convert to JSONL format

# Generate file name and save the JSONL data
training_data_filename = f"tune_data_stack_overflow_python_qa-{date}.jsonl"
with open(training_data_filename, "w") as f:
    f.write(tune_jsonl)


Conclusion: Comprehensive Overview
In this notebook, we demonstrated the complete process of setting up a machine learning environment to explore, prepare, and optimize a large dataset for tuning a foundation language model. The focus was on handling large datasets (from a public Stack Overflow dataset) in a scalable and efficient manner, ensuring that even when datasets do not fit into memory, we can process and train on them without issues. Below is a summary of each step:

1. Project Environment Setup
We initialized the environment by loading necessary credentials and libraries, specifically focusing on Google Cloud Vertex AI and BigQuery to manage large-scale data and machine learning tasks.
Vertex AI helps with running and scaling machine learning workflows, while BigQuery allows us to query large datasets efficiently.
2. BigQuery Data Access and Exploration
We explored the Stack Overflow public dataset hosted on BigQuery, which contains millions of questions, answers, and metadata related to Stack Overflow interactions.
Using SQL queries, we retrieved a list of table names and explored the first few rows from the posts_questions table to understand the structure and contents of the data.
3. Handling Large Datasets
Since the dataset is extremely large (potentially too big to fit into memory), we implemented strategies to manage such data:
Query Optimization: We limited our data exploration to manageable chunks using the SQL LIMIT clause.
Error Handling: We ensured that if we attempted to load an oversized dataset, the program could gracefully handle it by catching exceptions.
4. Joining Tables for Input and Output Creation
To prepare data for training a foundation model, we joined the questions (from posts_questions) with their corresponding answers (from posts_answers), filtering the data to focus on questions tagged with "Python" and posted after 2020.
This step also optimized the query to return only a specific number of rows (10,000), ensuring the retrieval was efficient without overloading memory or computational resources.
5. Adding Instructions for Language Model Fine-Tuning
We enhanced the dataset by adding instructions for the Language Model. Adding these instructions significantly improves the performance of large language models (LLMs), as it provides explicit context about the task to be performed.
The model is prompted to behave like a Stack Overflow developer answering Python-related questions. This approach is based on research that shows LLMs perform better when given clear task-specific instructions.
6. Dataset Preparation for Fine-Tuning
After preparing the questions and answers (now enhanced with instructions), we divided the dataset into training and evaluation sets, using an 80/20 split.
This allows for more data to be used for tuning the model while holding out 20% of the data for evaluation, ensuring that the model's performance is tested on unseen data.
7. Data Versioning and Storage
Versioning data is crucial for reproducibility, ensuring that experiments can be tracked and models can be re-trained or improved with specific datasets.
We generated a timestamped JSONL (JSON Lines) file, which is the ideal format for LLM training tasks. This file contains the input questions (with instructions) and the corresponding answers, ready to be fed into a model for fine-tuning.
Key Takeaways:
Scalable Data Processing: By leveraging tools like BigQuery and Vertex AI, we were able to handle large-scale datasets efficiently without running into memory limitations. This is crucial for real-world applications, where datasets often exceed local memory capabilities.

Query Optimization and Data Retrieval: When working with massive datasets, careful query planning, filtering, and limiting results help retrieve data without overwhelming the system.

Instruction-Based Model Tuning: Adding clear instructions to the input data can improve model performance, especially in scenarios where the model needs to generalize to unseen tasks or specific domains (in this case, Python programming questions).

Dataset Splitting and Versioning: Splitting the dataset into training and evaluation sets ensures fair testing, and versioning with timestamps helps maintain traceability across different experiments or model versions.

Next Steps:
The dataset is now ready for fine-tuning a language model. The next steps would include:

Uploading the prepared JSONL file to a cloud storage bucket.
Training a foundation model using the prepared dataset by fine-tuning it on Vertex AI or another machine learning service.
Monitoring and evaluating the model's performance on the evaluation set, and making iterative improvements based on the results.